In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import random
import pandas as pd
import numpy as np
import re

In [2]:
from datetime import datetime, timedelta

start_date = datetime(2010, 1, 1) 
end_date = datetime(2024, 10, 30)

date_list = []
current_date = start_date

while current_date <= end_date:
    date_list.append(current_date.strftime("%Y-%m-%d"))
    current_date += timedelta(1)

print(date_list)

['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08', '2010-01-09', '2010-01-10', '2010-01-11', '2010-01-12', '2010-01-13', '2010-01-14', '2010-01-15', '2010-01-16', '2010-01-17', '2010-01-18', '2010-01-19', '2010-01-20', '2010-01-21', '2010-01-22', '2010-01-23', '2010-01-24', '2010-01-25', '2010-01-26', '2010-01-27', '2010-01-28', '2010-01-29', '2010-01-30', '2010-01-31', '2010-02-01', '2010-02-02', '2010-02-03', '2010-02-04', '2010-02-05', '2010-02-06', '2010-02-07', '2010-02-08', '2010-02-09', '2010-02-10', '2010-02-11', '2010-02-12', '2010-02-13', '2010-02-14', '2010-02-15', '2010-02-16', '2010-02-17', '2010-02-18', '2010-02-19', '2010-02-20', '2010-02-21', '2010-02-22', '2010-02-23', '2010-02-24', '2010-02-25', '2010-02-26', '2010-02-27', '2010-02-28', '2010-03-01', '2010-03-02', '2010-03-03', '2010-03-04', '2010-03-05', '2010-03-06', '2010-03-07', '2010-03-08', '2010-03-09', '2010-03-10', '2010-03-11', '2010-03-12', '2010

In [ ]:
#https://finance.naver.com/news/mainnews.naver?date=2024-11-03&page=3
size = len(date_list)
base_url = "https://finance.naver.com/news/mainnews.naver"
titles_by_date = [[] for i in range(size)]
for i, date in enumerate(date_list):
    print(date)
    page = 1
    while (page <= 5): #10페이지 미만인 것을 어떻게 할까
        url = f"{base_url}?date={date}&page={page}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(url, headers= headers)

        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        news_items = soup.select('div.mainNewsList._replaceNewsLink ul.newsList li.block1 dd.articleSubject a')

        if not news_items:
            break

        for news in news_items:
            title = news.get_text(strip = True)
            titles_by_date[i].append(title)

        
        page += 1  

In [ ]:
# 텍스트 파일로 저장
for i,data in enumerate(titles_by_date):
    with open(f"{date_list[i]}.txt", "w") as file:
        for news in data:
            file.write(news + "\n") 

In [138]:
zero_list = pd.Series(titles_by_date)[list(map(lambda x : len(x)==0, titles_by_date))].reset_index()['index']
zero_list = list(pd.Series(date_list)[zero_list])

In [ ]:
#https://finance.naver.com/news/mainnews.naver?date=2024-11-03&page=3

#증권뉴스 크롤링 후 날짜 이름으로 된 파일에 저장.

base_url = "https://finance.naver.com/news/mainnews.naver"
for i, date in tqdm(enumerate(zero_list)):
    print(date)
    with open(f"{zero_list[i]}.txt", "w") as file:
        page = 1
        while (page <= 5): #10페이지 미만인 것을 어떻게 할까 
            url = f"{base_url}?date={date}&page={page}"
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
            }
            response = requests.get(url, headers= headers)

            if response.status_code != 200:
                break

            soup = BeautifulSoup(response.text, 'html.parser')
            news_items = soup.select('div.mainNewsList._replaceNewsLink ul.newsList li.block1 dd.articleSubject a')

            if not news_items:
                break

            for news in news_items:
                title = news.get_text(strip = True)
                file.write(title + "\n") 
        
            page += 1  

In [ ]:
# 중복된 뉴스 제거하기.

for date in date_list:

	# 파일을 읽고 수정하기
	with open(f'{date}.txt', 'r') as file:
		# 파일의 모든 내용을 읽습니다
		data = file.read()

	with open(f'{date}.txt', 'w') as file:
		for news in list(set(data.splitlines())):
			file.write(news + '\n')

In [22]:
# 각 파일 앞의 '[]' 파트 없애기
for date in date_list:
    with open(f'{date}.txt', 'r') as file:
        # 파일의 모든 내용을 읽습니다
        data = file.read()
        
    cleaned_data = re.sub(r"\[.*?\]", "", data)
    with open(f'{date}.txt', 'w') as file:
        for news in list(set(cleaned_data.splitlines())):
            file.write(news + '\n')

### 추가적으로 필요한 데이터 전처리

1. 키워드 추출할 때, 그날의 이슈를 잘 나타낼 수 있는 단어를 고르는 방법이 무엇일까?


In [3]:
# 데이터가 준비되었으니 keyword 추출 코드
# 1. text 토큰화.

from transformers import BertModel, BertTokenizer
import torch

# KoBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

data_keyword = []

for date in date_list:
    print(date)
    with open(f'{date}.txt', 'r') as file:
        data = file.read()

    if not data:
        data_keyword.append([])
        continue

    corpus = []
    data_new = re.sub(r"\n", ",", data)
    corpus.append(data_new)

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)

    # 각 단어의 중요도 추출
    tfidf_scores = X.toarray()[0] # 첫 번째 뉴스기사 제목의 키워드.
    feature_names = vectorizer.get_feature_names_out()

    # 높은 TF-IDF 스코어 순으로 정렬하여 상위 키워드 추출
    sorted_indices = tfidf_scores.argsort()[::-1]
    top_keywords = [feature_names[i] for i in sorted_indices[:5]]  # 상위 5개 키워드
    data_keyword.append(top_keywords)

data_keyword

2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05
2010-01-06
2010-01-07
2010-01-08
2010-01-09
2010-01-10
2010-01-11
2010-01-12
2010-01-13
2010-01-14
2010-01-15
2010-01-16
2010-01-17
2010-01-18
2010-01-19
2010-01-20
2010-01-21
2010-01-22
2010-01-23
2010-01-24
2010-01-25
2010-01-26
2010-01-27
2010-01-28
2010-01-29
2010-01-30
2010-01-31
2010-02-01
2010-02-02
2010-02-03
2010-02-04
2010-02-05
2010-02-06
2010-02-07
2010-02-08
2010-02-09
2010-02-10
2010-02-11
2010-02-12
2010-02-13
2010-02-14
2010-02-15
2010-02-16
2010-02-17
2010-02-18
2010-02-19
2010-02-20
2010-02-21
2010-02-22
2010-02-23
2010-02-24
2010-02-25
2010-02-26
2010-02-27
2010-02-28
2010-03-01
2010-03-02
2010-03-03
2010-03-04
2010-03-05
2010-03-06
2010-03-07
2010-03-08
2010-03-09
2010-03-10
2010-03-11
2010-03-12
2010-03-13
2010-03-14
2010-03-15
2010-03-16
2010-03-17
2010-03-18
2010-03-19
2010-03-20
2010-03-21
2010-03-22
2010-03-23
2010-03-24
2010-03-25
2010-03-26
2010-03-27
2010-03-28
2010-03-29
2010-03-30
2010-03-31
2010-04-01

[['환율', '달렸다', '끝났다', '남겨진', '내지말고'],
 ['외국인', '매매동향', '기관', '2010년', '최고성적'],
 ['새해', '혁명', '해야', '소비', '생보사주'],
 ['상승', '1월', '증시', '주가', '2009년'],
 ['증시', '불안한', '환율하락에', '2004년과', '늘릴까'],
 ['외국인', '해외기업들도', '베팅', '3色', '개인'],
 ['실적', '효자의', '삼성', '바닥', '매수'],
 ['강세', '한국인은', '넘었다', '상승', '삼성전자'],
 ['증시', '일자리와', '소비가', '변수다', '2010'],
 ['증시', '황제주', '돌아가며', '손에', '선샤인'],
 ['이어', '횡보장세', '고용악화에', '뚫을까', '떨어질까'],
 ['기대감', '힘들다', '신호탄', '속도', '새해'],
 ['조선', '흔들리는', '도입', '여파로', '신세'],
 ['훈풍될까', '삼성생명', '모멘텀', '무난히', '버릇'],
 ['인텔', '포스코', '상승', '호랑이해마다', '매수기회'],
 ['한다', '코스피', '15', '기관', '단단히'],
 ['1700선', '산업', '발표', '동향', '대응을'],
 ['현실', '급증', '상승', '삼성생명', '불구하고'],
 ['돌파', '노려라', '한국에', '시총', '소외종목'],
 ['상승', '삼성생명', '들썩', '효과', '기업경기'],
 ['화두', '랠리', '누를까', '늘것', '달러강세에'],
 ['한국기술산업', '서울통신기술', '비실비실', '베이비붐', '매물벽'],
 ['키워드는', '노린다', '22', '3d', 'tv로'],
 ['종목은', '휘청인', '악재', '실적', '상승'],
 ['증시', '경신', '주가', '코스피', '힘들듯'],
 ['급락', '실적', '환율', '확실한건', '85p'],
 ['기회', '지수', '외국인', 

In [ ]:
tokenized_keyword = []

# 토큰화
for i,data in enumerate(data_keyword):
    print(date_list[i])
    if not data : 
        tokenized_keyword.append(torch.zeros(5,5,model.config.hidden_size))
        continue
    inputs = tokenizer(data, return_tensors="pt", padding="max_length", max_length=5, truncation=True)
    # KoBERT로 임베딩 생성
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        tokenized_keyword.append(embeddings)

tokenized_keyword


In [ ]:
## Getting KOSPI INDEX DATA:

from pykrx import stock

# 코스피 지수 데이터 가져오기
kospi_data = stock.get_index_ohlcv_by_date("20100101", "20241031", "1001")  # 1001은 코스피의 ID

kospi_price_data = pd.DataFrame(kospi_data['종가']) 
# 날짜가 다른 문제 발생

In [47]:
rtrn = [1]+[kospi_price_data.종가[i] / kospi_price_data.종가[i-1] for i in range(1,3656)] 
kospi_price_data['rtrn'] = rtrn
kospi_price_data['log_return'] = np.log(rtrn)
kospi_price_data['volatility'] = (kospi_price_data.log_return)**2
kospi_price_data

종가      rtrn  log_return    volatility
날짜                                                     
2010-01-04  1696.14  1.000000    0.000000  0.000000e+00
2010-01-05  1690.62  0.996746   -0.003260  1.062601e-05
2010-01-06  1705.32  1.008695    0.008657  7.495146e-05
2010-01-07  1683.45  0.987175   -0.012908  1.666040e-04
2010-01-08  1695.26  1.007015    0.006991  4.887216e-05
...             ...       ...         ...           ...
2024-10-25  2583.27  1.000868    0.000867  7.525462e-07
2024-10-28  2612.43  1.011288    0.011225  1.259958e-04
2024-10-29  2617.80  1.002056    0.002053  4.216647e-06
2024-10-30  2593.79  0.990828   -0.009214  8.490045e-05
2024-10-31  2556.15  0.985488   -0.014618  2.136832e-04

[3656 rows x 4 columns]

In [58]:
# 주식데이터 날짜에 맞도록 뉴스데이터 날짜를 재정렬 시킨다. 
# 예를 들어, 주말의 뉴스는 금요일의 주식 뉴스로 편입 시킨다.
# 또한, notation의 편의를 위해, 학습데이터의 제목은, 그 예측대상 날짜를 기준으로 한다. 즉 1031일의 변동성을 예측하기 위한 데이터의 제목을 1031.txt로 생각한다.
kospi_date_list = list(map(lambda x: x[:10], list(map(str, kospi_price_data.index))))



True